In [ ]:
import pdal
import json
import os
from urban3d.trees import *
from glob import glob
# Method 3: Create a numpy array stack
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
os.environ['OMP_NUM_THREADS'] = '3'

ROOT = "../../"
PROJECT_NAME = "GTA_2023/"


DATA = os.path.join(ROOT, 'data/')
PROJECT_PATH = os.path.join(DATA , PROJECT_NAME)
RAW = os.path.join(PROJECT_PATH, 'raw/')
FILTERED = os.path.join(PROJECT_PATH, 'filtered/')
CLASSIFIED = os.path.join(PROJECT_PATH, 'classified/')
MERGED = os.path.join(PROJECT_PATH, 'merged/')
MESH = os.path.join(PROJECT_PATH, 'mesh/')

# INPUT FILE PATHS
folder = os.path.join(RAW , "LAZ/" )
las_name = "ON_TRCA2023_20230511_NAD83CSRS_UTMZ17_1km_E6290_N48340_CLASS.copc.laz"
in_las = os.path.join(folder, las_name)
grid_id = '_'.join(las_name.split('_')[-3:-1])

## Filter Raw Point Cloud

In [ ]:
# ## Execute Tree Filtering Pipeline
tree_pp= pdal_tree_filter(in_las, FILTERED, count_limit=800000)
tree_pp.execute()

3228463

## Isolate Trees + Further Filtering

In [ ]:
## Filter out Cluster ID with Trees
filtered_las = [os.path.join(FILTERED,file) for file in os.listdir(FILTERED) if grid_id in file]
classified_las = [os.path.join(CLASSIFIED,file.replace('.copc.laz', '_treeiso.copc.laz')) for file in os.listdir(FILTERED) if grid_id in file]
laspy.read(in_las).header.parse_crs()

## FILTERING BY CLUSTER ID AND HEIGHT
for in_las, out_las in zip(filtered_las, classified_las):
    las = laspy.read(in_las)
    dimensions = ['X', 'Y', 'Z', 'ClusterID','intensity', 'number_of_returns', 'Curvature', ]

    # ## Filter by Cluster ID
    df = build_las_df(las, dimensions, rescale=True)
    cluster_id = select_cluster(df)
    las = las[las.ClusterID == cluster_id]

    
    # ## Filter By Height Above Ground
    hag = las.X
    mean_hag = np.mean(hag)
    std_hag =  np.std(hag)

    hag_mask = hag < (std_hag * 3)
    las = las[hag_mask]

    # ## Write LAS
    las.write(in_las)


# ## TREE ISOLATION AND FINAL K MEANS FILTER
for in_las, out_las in zip(filtered_las, classified_las):

    # ## Isolate Trees
    process_las_file(in_las, out_las, if_isolate_outlier=False)

    ## Second Kmeans Pass to filter out Trees from Tree ISO Segments
    las = laspy.read(out_las)
    dimensions = ['X', 'Y', 'Z', 'final_segs','intensity', 'number_of_returns', 'Curvature',  ]
    df = build_las_df(las, dimensions, rescale=True)
    df['final_segs'] = las.final_segs
    df_mean = df.groupby('final_segs').mean().reset_index()
    df_mean['count'] = df.groupby('final_segs').X.count()

    kmeans = KMeans(n_clusters= 2)
    df_mean['ClusterID'] = kmeans.fit_predict(df_mean[['Curvature', 'intensity']])
    df_mean = df_mean[df_mean.ClusterID == select_cluster(df_mean)]

    q_lower =  df_mean['count'].quantile(q=0.3)
    q_upper =  df_mean['count'].quantile(q=0.99)
    df_mean = df_mean[(df_mean['count'] > q_lower) & (df_mean['count'] < q_upper)] 

    ## Out put Cleaned Las with Trees
    las_clean = las[df.final_segs.isin(df_mean.final_segs).values]
    las_clean.write(out_las)

*******Processing LAS/LAZ******* ../data/GTA_2023/filtered/ON_TRCA2023_20230511_NAD83CSRS_UTMZ17_1km_E6290_N48340_CLASS_filtered_1.copc.laz


d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\treeiso\treeiso.py:318: RuntimeWarning: divide by zero encountered in scalar divide
  currentGroupRelHt = (currentGroupFt[0] - np.min(nnGroupFt[:, 0])) / currentGroupFt[1]
d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\treeiso\treeiso.py:108: RuntimeWarning: invalid value encountered in scalar divide
  intersection_area / mask2_area
d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\treeiso\treeiso.py:107: RuntimeWarning: invalid value encountered in scalar divide
  intersection_area / mask1_area,


*******End processing*******


d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\trees.py:159: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.ClusterID = las.ClusterID
c:\Users\USER\anaconda3\envs\geo\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


*******Processing LAS/LAZ******* ../data/GTA_2023/filtered/ON_TRCA2023_20230511_NAD83CSRS_UTMZ17_1km_E6290_N48340_CLASS_filtered_2.copc.laz


...

d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\trees.py:159: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.ClusterID = las.ClusterID
c:\Users\USER\anaconda3\envs\geo\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


*******Processing LAS/LAZ******* ../data/GTA_2023/filtered/ON_TRCA2023_20230511_NAD83CSRS_UTMZ17_1km_E6290_N48340_CLASS_filtered_5.copc.laz


d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\treeiso\treeiso.py:318: RuntimeWarning: divide by zero encountered in scalar divide
  currentGroupRelHt = (currentGroupFt[0] - np.min(nnGroupFt[:, 0])) / currentGroupFt[1]
d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\treeiso\treeiso.py:108: RuntimeWarning: invalid value encountered in scalar divide
  intersection_area / mask2_area
d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\treeiso\treeiso.py:107: RuntimeWarning: invalid value encountered in scalar divide
  intersection_area / mask1_area,


*******End processing*******


d:\05_Projects\01_Active\Toronto Digital Twin\pipeline3d\src\trees.py:159: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.ClusterID = las.ClusterID
c:\Users\USER\anaconda3\envs\geo\Lib\site-packages\sklearn\cluster\_kmeans.py:1419: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


##  Merge

In [31]:
out_merged = os.path.join(MERGED, las_name.replace('.copc.laz', '_merged.las'))


p = classified_las.copy()
p.extend([{"type": "filters.merge"}, {"type" : "writers.las", "filename" : out_merged}])
p = dict(pipeline = p)
p = pdal.Pipeline(json.dumps(p))
p.execute()


1138825